In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')
gender_submission=pd.read_csv("gender_submission.csv")

In [4]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [8]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
#drop PassengerId, Name, Ticket
train=train.drop(['PassengerId','Name','Ticket'],axis=1)
test=test.drop(['Name','Ticket'],axis=1)

In [16]:
#count missing values
train.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [11]:
#drop cabin column
train=train.drop(['Cabin'],axis=1)
test=test.drop(['Cabin'],axis=1)

In [15]:
#drop rows with missing values
train=train.dropna()


In [17]:
#train test split, target column name=Survived

X=train.drop(['Survived'],axis=1)
y=train['Survived']

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)



In [21]:
#check data types
X_train.dtypes

Pclass          int64
Age           float64
SibSp           int64
Parch           int64
Fare          float64
Sex_female       bool
Sex_male         bool
Embarked_C       bool
Embarked_Q       bool
Embarked_S       bool
dtype: object

In [20]:
#convert object data types to one hot encoding
X_train=pd.get_dummies(X_train)
X_test=pd.get_dummies(X_test)
test=pd.get_dummies(test)


In [24]:
#making a xgboost model
from xgboost import XGBClassifier
model=XGBClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

#accuracy
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)


0.7832167832167832

In [33]:
#optimizing hyperparameters using grid search
from sklearn.model_selection import GridSearchCV
param_grid={
    'learning_rate':[0.01,0.1,0.2,0.3],
    'max_depth':[3,4,5,6],
    'min_child_weight':[1,2,3,4],
    'gamma':[0.0,0.1,0.2,0.3],
    'colsample_bytree':[0.3,0.4,0.5,0.6]
}
grid=GridSearchCV(model,param_grid,n_jobs=-1,cv=5,verbose=3)
grid.fit(X_train,y_train)
grid.best_params_

#using best parameters
model=XGBClassifier(colsample_bytree=0.4,gamma=0.2,learning_rate=0.1,max_depth=4,min_child_weight=1)
model.fit(X_train,y_train)
y_pred=model.predict(X_test)
accuracy_score(y_test,y_pred)


Fitting 5 folds for each of 1024 candidates, totalling 5120 fits
[CV 4/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.01, max_depth=3, min_child_weight=1;, score=0.781 total time=   0.0s
[CV 5/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.01, max_depth=3, min_child_weight=2;, score=0.796 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.01, max_depth=3, min_child_weight=1;, score=0.816 total time=   0.0s
[CV 2/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.01, max_depth=3, min_child_weight=2;, score=0.789 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.01, max_depth=3, min_child_weight=1;, score=0.798 total time=   0.0s
[CV 1/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.01, max_depth=3, min_child_weight=2;, score=0.798 total time=   0.0s
[CV 3/5] END colsample_bytree=0.3, gamma=0.0, learning_rate=0.01, max_depth=3, min_child_weight=2;, score=0.833 total time=   0.0s
[CV 3/5] END colsa

0.7832167832167832

In [34]:
#making logistic regression model
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

#accuracy
accuracy_score(y_test,y_pred)

/Users/krishan/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8041958041958042

In [29]:
test_x=test.drop(['PassengerId'],axis=1)



In [30]:
#predicting test data
y_pred=model.predict(test_x)

In [31]:
#creating submission file
submission=pd.DataFrame({'PassengerId':test['PassengerId'],'Survived':y_pred})

In [32]:
submission.to_csv('submission.csv',index=False)